In [8]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen
import joblib


In [ ]:
## Get Cacao-trained permeability model
permeability_model = joblib.load("../models/permeability_rf.joblib")

In [20]:
## Get Cacao permeability data
from tdc.single_pred import ADME
cacao_data = ADME(name='Caco2_Wang')

Found local copy...
Loading...
Done!


In [41]:
# --- CONFIG ---
import os

os.environ["DB_USER"] = "postgres"
os.environ["DB_PASS"] = "AzuleneLabs_2026"
os.environ["DB_HOST"] = "azulene-1.cizeysmsgxmm.us-east-1.rds.amazonaws.com"
os.environ["DB_NAME"] = "postgres"
os.environ["DB_PORT"] = "5432"

print(os.getenv("DB_USER"))  # confirm

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_PORT = os.getenv("DB_PORT", 5432)

BATCH_SIZE = 1000

def get_cacao_permeability(df, cacao):
    """Fetch permeability from CACAO database (placeholder function)."""
    # ## add all permeability data to df 
    ## 'Drug' column in cacao == 'smiles' column in df, smiles might overlap, or might nowt be present at all

    cacao = cacao.rename(columns={"Drug": "smiles", "Y": "cacao_permeability"})

    # Merge on 'smiles' — keep all molecules from df
    merged = df.merge(cacao[["smiles", "cacao_permeability"]], on="smiles", how="left")


    return merged



# --- Connect to PostgreSQL ---
conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(conn_str)


# --- RDKit-based calculations ---
def compute_rdkit_features(smiles):
    """Compute molecular weight, logP, etc. using RDKit."""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            return None
        return {
            "mol_weight": Descriptors.MolWt(mol),
            "logp_rdkit": Crippen.MolLogP(mol)
        }
    except Exception:
        return None


# --- PubChem fallback API ---
def fetch_pubchem_logp(smiles):
    """Fetch experimental logP from PubChem."""
    try:
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{smiles}/property/LogP,IsomericSMILES/JSON"
        resp = requests.get(url, timeout=5)
        if resp.status_code == 200:
            props = resp.json().get("PropertyTable", {}).get("Properties", [{}])[0]
            return props.get("LogP")
    except Exception:
        return None


# --- Load data ---
def fetch_data(limit=None):
    """Fetch data from PostgreSQL table."""
    query = "SELECT * FROM drug_properties"
    if limit:
        query += f" LIMIT {limit}"
    return pd.read_sql(query, engine)


def fetch_data_enriched(limit=None):
    """Fetch data from PostgreSQL table."""
    query = "SELECT * FROM drug_properties_enriched"
    if limit:
        query += f" LIMIT {limit}"
    return pd.read_sql(query, engine)


# --- Enrichment Pipeline ---
def enrich_dataframe(df):

    df = df.dropna(subset=["smiles"]).copy()
    df = df[df["smiles"].apply(lambda s: isinstance(s, str) and len(s.strip()) > 0)]


    ## Get the caco permeability data from cacao and merge into df
    cacao = cacao_data.get_data()
    print(cacao.head())
    print(len(df))
    df = get_cacao_permeability(df, cacao)
    print(len(df))
    
    enriched = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Enriching molecules"):
        smiles = row.get("smiles")

        mol = Chem.MolFromSmiles(smiles)
        logp = Descriptors.MolLogP(mol)
        psa = Descriptors.TPSA(mol)
        mw = Descriptors.MolWt(mol)
        hbd = Descriptors.NumHDonors(mol)
        hba = Descriptors.NumHAcceptors(mol)

        desc = [
            mw,
            logp,
            psa,
            hbd,
            hba,
            Descriptors.NumRotatableBonds(mol)
        ]
        
        ## Include permeability model prediction AND direct predictions from CACAO when available (NA if othterwise)
        #permeability = get_cacao_permeability(df, cacao)
        permeability_predictions = float(permeability_model.predict([desc])[0])
        row["predicted_permeability"] = permeability_predictions

        row["hba"]  = hba
        row["hbd"]  = hbd
        row["psa"]  = psa
        row["molecular_weight"] = mw



        data_origin = {}

        if not smiles:
            enriched.append(row)
            continue

        # --- Compute with RDKit ---
        rdkit_features = compute_rdkit_features(smiles)
        if rdkit_features:
            if pd.isna(row.get("logp")) and rdkit_features["logp_rdkit"] is not None:
                row["logp"] = rdkit_features["logp_rdkit"]
                data_origin["logp"] = "rdkit"
            if pd.isna(row.get("binding_free_energy")):
                # (placeholder example)
                row["binding_free_energy"] = -0.1 * rdkit_features["logp_rdkit"]
                data_origin["binding_free_energy"] = "estimated_rdkit"

        # --- PubChem fallback ---
        if pd.isna(row.get("logp")):
            logp_pubchem = fetch_pubchem_logp(smiles)
            if logp_pubchem is not None:
                row["logp"] = logp_pubchem
                data_origin["logp"] = "pubchem"

        # --- Simple pKa estimation (toy model) ---
        if pd.isna(row.get("pka")) and rdkit_features:
            row["pka"] = 7.0 - 0.2 * rdkit_features["logp_rdkit"]
            data_origin["pka"] = "estimated_rdkit"

        # --- Solubility fallback (basic logS estimation) ---
        if pd.isna(row.get("solubility")) and rdkit_features:
            logp = rdkit_features["logp_rdkit"]
            molwt = rdkit_features["mol_weight"]
            row["solubility"] = -0.01 * molwt - 0.5 * logp
            data_origin["solubility"] = "estimated_rdkit"

        # Track origin of each field
        row["metadata"] = json.dumps({"data_origin": data_origin})
        enriched.append(row)

    return pd.DataFrame(enriched)


postgres


In [22]:
original_df = fetch_data()
print(f"Fetched {len(original_df)} records.")
print(original_df.head())

Fetched 29982 records.
      chembl_id                                            smiles  \
0    CHEMBL6329      Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl   
1    CHEMBL6328   Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1   
2  CHEMBL265667  Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1   
3    CHEMBL6362      Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1   
4  CHEMBL267864    Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1   

  binding_free_energy  solubility  logp permeability   pka  molecular_weight  \
0                None         NaN   NaN         None  None               NaN   
1                None         NaN   NaN         None  None               NaN   
2                None         NaN   NaN         None  None               NaN   
3                None         NaN   NaN         None  None               NaN   
4                None         NaN   NaN         None  None               NaN   

   hba  hbd  psa  rtb qed_weighted  source  \
0  NaN  NaN  NaN  NaN         None 

In [23]:
enriched_df = enrich_dataframe(original_df)
print(enriched_df.head())

                                             Drug_ID  \
0                                    (-)-epicatechin   
1  (2E,4Z,8Z)-N-isobutyldodeca-2,4,10-triene-8 -y...   
2                                            codeine   
3                                         creatinine   
4                                            danazol   

                                                Drug         Y  
0            Oc1cc(O)c2c(c1)OC(c1ccc(O)c(O)c1)C(O)C2 -6.220000  
1                   C/C=C\C#CCC/C=C\C=C\C(=O)NCC(C)C -3.860000  
2  COc1ccc2c3c1O[C@H]1[C@@H](O)C=C[C@H]4[C@@H](C2... -4.090000  
3                                     CN1CC(=O)NC1=N -5.935409  
4  C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=Cc5oncc5C[C@]4(... -4.840000  
29860
29861


Enriching molecules:  68%|██████▊   | 20194/29861 [06:29<02:35, 62.11it/s][09:34:10] WARNING: not removing hydrogen atom without neighbors
[09:34:11] WARNING: not removing hydrogen atom without neighbors
Enriching molecules:  68%|██████▊   | 20325/29861 [06:31<02:33, 61.99it/s][09:34:13] WARNING: not removing hydrogen atom without neighbors
[09:34:13] WARNING: not removing hydrogen atom without neighbors
[09:34:13] WARNING: not removing hydrogen atom without neighbors
[09:34:13] WARNING: not removing hydrogen atom without neighbors
Enriching molecules:  69%|██████▊   | 20520/29861 [06:34<02:49, 54.96it/s][09:34:16] WARNING: not removing hydrogen atom without neighbors
[09:34:16] WARNING: not removing hydrogen atom without neighbors
Enriching molecules:  69%|██████▉   | 20581/29861 [06:35<02:32, 60.89it/s][09:34:17] WARNING: not removing hydrogen atom without neighbors
[09:34:17] WARNING: not removing hydrogen atom without neighbors
Enriching molecules:  69%|██████▉   | 20684/29861 [06:

      chembl_id                                            smiles  \
0    CHEMBL6329      Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl   
1    CHEMBL6328   Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1   
2  CHEMBL265667  Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1   
3    CHEMBL6362      Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1   
4  CHEMBL267864    Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1   

   binding_free_energy  solubility     logp permeability       pka  \
0            -0.211362    -4.47435  2.11362         None  6.577276   
1            -0.133190    -3.98914  1.33190         None  6.733620   
2            -0.227274    -4.71434  2.27274         None  6.545452   
3            -0.146022    -3.80320  1.46022         None  6.707956   
4            -0.211362    -4.47435  2.11362         None  6.577276   

   molecular_weight  hba  hbd     psa  rtb qed_weighted  source  \
0           341.754    5    1   84.82  NaN         None  ChEMBL   
1           332.319    6    1 

In [25]:
enriched_df[['cacao_permeability', 'predicted_permeability']].head()

,cacao_permeability,predicted_permeability
0,NaN,-4.397749
1,NaN,-4.659045
2,NaN,-4.839266
3,NaN,-4.536761
4,NaN,-4.397749


In [32]:
enriched_df['predicted_permeability'].value_counts(dropna=False) ## No NaNs!!

predicted_permeability
-6.775548    124
-4.501009     38
-4.434505     33
-4.699744     33
-4.617371     24
            ... 
-4.964521      1
-4.591178      1
-6.143881      1
-4.978256      1
-4.365837      1
Name: count, Length: 24608, dtype: int64

In [30]:
import numpy as np
## Compare cacao perm with predicted perm
just_perm = enriched_df[['cacao_permeability', 'predicted_permeability']]
just_perm_no_nan = just_perm.dropna()
just_perm_no_nan["Error"] = np.abs(just_perm_no_nan["cacao_permeability"] - just_perm_no_nan["predicted_permeability"])

just_perm_no_nan.head()

,cacao_permeability,predicted_permeability,Error
75,-4.987000,-4.957837,0.029163
147,-4.690000,-4.736457,0.046457
183,-4.809974,-4.727885,0.082089
322,-4.958607,-5.094805,0.136198
518,-4.851000,-4.441104,0.409896


In [31]:
print(just_perm_no_nan.describe()) ## Difference between cacao and predicted permeability is small

       cacao_permeability  predicted_permeability      Error
count           99.000000               99.000000  99.000000
mean            -4.814975               -4.846693   0.152299
std              0.632247                0.540720   0.145915
min             -7.380000               -7.052423   0.005595
25%             -4.989200               -5.013632   0.054090
50%             -4.700000               -4.741600   0.111777
75%             -4.440000               -4.502791   0.210597
max             -3.510000               -3.966881   0.910420


## Adding Binding Free Energies (derive from CHEMBL AND predict using SMILES)

In [35]:
## Try TDC

# Correct import for multi-instance prediction:
from tdc.multi_pred import DTI

# Then, access the specific BindingDB dataset by name
data = DTI(name='BindingDB_Kd')  # For datasets with Kd units
binding_db = data.get_data()
print(binding_db.head())


R = 1.987e-3  # kcal/mol·K
T = 298
binding_db['binding_db_bfe'] = R * T * np.log(binding_db['Y'] * 1e-9)  # Kd (nM → M)
binding_db.rename(columns={"Drug": "smiles"}, inplace=True)

binding_db.head()

Downloading...
100%|██████████| 54.4M/54.4M [00:05<00:00, 9.46MiB/s]
Loading...
Done!


    Drug_ID                                            Drug Target_ID  \
0  444607.0       Cc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1    P00918   
1    4316.0      COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1    P00918   
2    4293.0           NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1    P00918   
3    1611.0    NS(=O)(=O)c1cc2c(s1)S(=O)(=O)N(Cc1cccs1)CC2O    P00918   
4    1612.0  COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1    P00918   

                                              Target     Y  
0  MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...  0.46  
1  MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...  0.49  
2  MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...  0.83  
3  MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...  0.20  
4  MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...  0.16  


,Drug_ID,smiles,Target_ID,Target,Y,binding_db_bfe
0,444607.0,Cc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.46,-12.730587
1,4316.0,COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.49,-12.693178
2,4293.0,NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.83,-12.381115
3,1611.0,NS(=O)(=O)c1cc2c(s1)S(=O)(=O)N(Cc1cccs1)CC2O,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.20,-13.223775
4,1612.0,COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.16,-13.355904


In [36]:
## Merge binding_db to enriched_df on smiles / Drug columns
enriched_data = enriched_df.merge(binding_db[['smiles', 'binding_db_bfe']], left_on='smiles', right_on='smiles', how='left', suffixes=('', '_tdc'))
enriched_data.head()

,chembl_id,smiles,binding_free_energy,solubility,logp,permeability,pka,molecular_weight,hba,hbd,psa,rtb,qed_weighted,source,metadata,cacao_permeability,predicted_permeability,binding_db_bfe
0,CHEMBL6329,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl,-0.211362,-4.47435,2.11362,None,6.577276,341.754,5,1,84.82,NaN,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre...",NaN,-4.397749,NaN
1,CHEMBL6328,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1,-0.133190,-3.98914,1.33190,None,6.733620,332.319,6,1,108.61,NaN,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre...",NaN,-4.659045,NaN
2,CHEMBL265667,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1,-0.227274,-4.71434,2.27274,None,6.545452,357.797,5,2,87.98,NaN,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre...",NaN,-4.839266,NaN
3,CHEMBL6362,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1,-0.146022,-3.80320,1.46022,None,6.707956,307.309,5,1,84.82,NaN,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre...",NaN,-4.536761,NaN
4,CHEMBL267864,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1,-0.211362,-4.47435,2.11362,None,6.577276,341.754,5,1,84.82,NaN,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre...",NaN,-4.397749,NaN


In [37]:
len(enriched_data)

32833

In [38]:
bfes = enriched_data[['binding_free_energy', 'binding_db_bfe']]
bfes_no_nan = bfes.dropna()
bfes_no_nan["Error"] = np.abs(bfes_no_nan["binding_free_energy"] - bfes_no_nan["binding_db_bfe"])
bfes_no_nan.head()

,binding_free_energy,binding_db_bfe,Error
75,-0.17846,-13.320006,13.141546
76,-0.17846,-13.374703,13.196243
77,-0.17846,-12.148206,11.969746
78,-0.17846,-11.725861,11.547401
79,-0.17846,-12.259059,12.080599


In [39]:
bfes_no_nan.describe()

,binding_free_energy,binding_db_bfe,Error
count,3241.000000,3241.000000,3241.000000
mean,-0.352681,-8.005819,7.653138
std,0.139869,1.944118,1.968566
min,-0.680320,-20.451308,2.519171
25%,-0.363330,-8.934279,6.476593
50%,-0.333494,-6.817103,6.496893
75%,-0.320210,-6.817103,8.622319
max,0.277780,-2.726841,20.330218


In [42]:
## Inspect enriched_data for predicted binding free energy values

bfe_enriched_data = fetch_data_enriched()
bfes = bfe_enriched_data[['binding_free_energy', 'binding_db_bfe']]
bfes = bfes.dropna()
bfes.head()

,binding_free_energy,binding_db_bfe
75,12.284173,-13.320006
76,12.284173,-13.374703
77,12.284173,-12.148206
78,12.284173,-11.725861
79,12.284173,-12.259059


In [43]:
## Ignore signs for now: find error between binding_free_energy and binding_db_bfe
bfes['Error'] = np.abs(bfes['binding_free_energy'] - np.abs(bfes['binding_db_bfe']))
bfes.head()

,binding_free_energy,binding_db_bfe,Error
75,12.284173,-13.320006,1.035833
76,12.284173,-13.374703,1.090529
77,12.284173,-12.148206,0.135967
78,12.284173,-11.725861,0.558312
79,12.284173,-12.259059,0.025115


In [ ]:
bfes.describe() ## shows that predicted binding free energy values are reasonable, only sign-inversed

,binding_free_energy,binding_db_bfe,Error
count,3241.000000,3241.000000,3241.000000
mean,7.967150,-8.005819,1.063772
std,1.158335,1.944118,1.103749
min,3.039430,-20.451308,0.000389
25%,7.324859,-8.934279,0.483420
50%,7.540588,-6.817103,0.723486
75%,8.435151,-6.817103,1.249155
max,12.889833,-2.726841,8.809638
